In [1]:
import pandas, nltk, string, numpy, csv
from pattern.fr import parse, split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import linalg
from sklearn.cluster import KMeans
import collections
from pprint import pprint

In [2]:
from langdetect import detect

In [3]:
df = pandas.read_pickle('API_Data_DataFrame')

In [4]:
df

,attributionURL,careerOpportunitiesRating,compensationAndBenefitsRating,cons,cultureAndValuesRating,currentJob,headline,industry,jobTitle,jobTitleFromDb,...,overallNumeric,overallRating,pros,ratingDescription,recommendToFriendRating,reviewDateTime,sectorName,seniorLeadershipRating,website,workLifeBalanceRating
0,http://www.glassdoor.com/Reviews/Employee-Revi...,2.9,2.6,Un moyen de deplacement pas tres pratique et f...,3.2,False,Goood,Federal Agencies,Facteur,Facteur,...,5,3.3,Un tres bon salaire pour un travail tres inter...,OK,0.6,2016-04-20 07:02:16.3,Government,2.8,www.laposte.fr,3.2
1,http://www.glassdoor.com/Reviews/Employee-Revi...,3.1,2.5,Le stress souvent present Le manque d'effectif...,3.0,True,Un super job d'appoint !,Fast-Food & Quick-Service Restaurants,Formateur,Formateur,...,4,3.2,Bonne ambianceSouplesse des planningsDu challe...,OK,0.7,2016-04-13 01:37:11.553,"Restaurants, Bars & Food Services",3.0,www.mcdonalds.com,2.6
2,http://www.glassdoor.com/Reviews/Employee-Revi...,3.4,3.5,Metiers de l'ingenieur moins valorises dans un...,3.5,True,Une entreprise internationale,Banks & Credit Unions,Employee,NA,...,4,3.5,Variete des projetsMoyens mis a disposition de...,Satisfied,0.8,2016-04-23 05:04:28.28,Finance,3.2,www.bnpparibas.com,3.6
3,http://www.glassdoor.com/Reviews/Employee-Revi...,3.1,3.5,noyes sous une multitude de filiales,3.7,True,Ingenieur systeme,Telecommunications Services,Employee,NA,...,4,3.6,"Bonne ambiance, manager competents et a l'ecoute",Satisfied,0.8,2016-04-20 23:55:18.127,Telecommunications,3.4,www.orange.com,3.9
4,http://www.glassdoor.com/Reviews/Employee-Revi...,2.9,3.4,Beaucoup de politique et de luttes internes. D...,3.2,True,Boite serieuse mais culture vieille ecole,IT Services,Employee,NA,...,4,3.4,Amadeus beneficie d'une excellente puissance d...,OK,0.6,2016-04-12 01:38:50.06,Information Technology,2.9,www.amadeus.com,4.0
5,http://www.glassdoor.com/Reviews/Employee-Revi...,3.3,3.3,Beaucoup de sollicitation et stressant,3.2,True,Un environnement propice au travail,Investment Banking & Asset Management,Employee,NA,...,5,3.4,Des bureaux agreable avec de nombreux avantages.,OK,0.8,2016-05-02 04:42:22.4,Finance,3.2,www.societegenerale.com,3.5
6,http://www.glassdoor.com/Reviews/Employee-Revi...,3.8,4.0,"pas de jours de conge, hormis si etablis par l...",3.7,False,Stagiaire marketing,Consumer Products Manufacturing,Stagiaire Marketing,Stagiaire Marketing,...,4,3.5,"En stage: acces aux ventes, formations",Satisfied,0.8,2016-04-30 10:06:05.227,Manufacturing,3.2,www.loreal.com,2.7
7,http://www.glassdoor.com/Reviews/Employee-Revi...,3.9,3.7,Necessite de passer rapidement d'une tache a u...,3.6,False,CDI temps partiel contrat etudiant,Grocery Stores & Supermarkets,Employee,NA,...,4,3.6,Journee de travail en continu avec pause dejeu...,Satisfied,0.9,2016-04-19 12:01:47.303,Retail,3.4,www.lidl.de,3.2
8,http://www.glassdoor.com/Reviews/Employee-Revi...,3.7,3.3,Une charge de travail tres importante,3.4,False,Belle entreprise,Consulting,Consultant Junior,Consultant Junior,...,4,3.6,Methodologie Accenture et un cadrage des missi...,Satisfied,0.8,2016-04-19 01:36:26.583,Business Services,3.0,www.accenture.com,2.4
9,http://www.glassdoor.com/Reviews/Employee-Revi...,3.3,3.5,Machine de guerre ! Employe non considere par ...,3.4,True,Rude !,General Merchandise & Superstores,Hotesse D'accueil/caisse,Hotesse D'accueil/caisse,...,4,3.6,"Avantages financiers, prime de vacance, 13eme ...",Satisfied,0.8,2016-04-24 10:23:12.827,Retail,3.2,www.carrefour.com,3.0


In [5]:
len(df)

5840

In [6]:
df['lang'] = df['pros'].apply(lambda x: detect(x))

In [7]:
df_fr = df[df['lang']=='fr']

In [8]:
pros = df_fr['pros']

In [9]:
df_fr['pros_lower'] = df_fr['pros'].apply(lambda x: x.lower())

C:\Users\elhamdani\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
df_fr['pros_tokens'] = df_fr['pros_lower'].apply(lambda x: parse(x.replace('.',""), chunks = False, lemmata = True).split()[0])

C:\Users\elhamdani\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
df_fr['pros_lemmas'] =\
df_fr['pros_tokens'].apply(lambda x: [t[2] for t in x if t[1] not in ['DT',\
'CC','IV','IN','PRP','PRP$','SYM','UH',',','.',':','(',')','-'] ])

C:\Users\elhamdani\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
df_fr['pros_lemmas'] = df_fr['pros_lemmas'].apply(lambda x: [t for t in x if t not in stopwords.words('french')])

C:\Users\elhamdani\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
df_fr['pros_j'] = df_fr['pros_lemmas'].apply(lambda x: " ".join(x))

C:\Users\elhamdani\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
len(df_fr['pros_j'])

4317

In [20]:
df_fr['pros_j']

0       tre bon salaire travail tre interessant contac...
1       bon ambiancesouplesse planningsdu challenge to...
2       variete projetsmoyen mettre avoir disposition ...
3             bon ambiance manager competent avoir ecoute
4       amadeu beneficie excellent puissance feu devel...
5                      bureau agreable nombreux avantages
6                               stage acce vent formation
7             journee travail continu pause dejeuner midi
8       methodologie accenture cadrage mission tre imp...
9       avantages financier prime vacance 13eme mois i...
10                                  prime tout trois mois
11      projet interessant innovant internebonne ambiance
12      bon ambiance travail equipe patron responsable...
13      beaucoup choses avoir apprendre industrie nucl...
14      beaucoup stagiaire vie environnement multicult...
15      salaire fourchette haute contrat apprentissage...
16      opportunite travailler grand projetsprise resp...
17      entrep

In [21]:
df_fr['pros'][6]

'En stage: acces aux ventes, formations'

# Création du Bag of Words

La suppression des mots comme 'je' à l'aide du POS n'est pas efficace. Le mieux est d'établire une liste des mots vides à supprimer. Pour cela on peut utiliser la liste de stop words de nltk

In [22]:
vectorizer = CountVectorizer(min_df=1, stop_words = stopwords.words('french'))

In [23]:
X = vectorizer.fit_transform(df_fr['pros_j'])

In [24]:
X

<4317x6809 sparse matrix of type '<type 'numpy.int64'>'
	with 48202 stored elements in Compressed Sparse Row format>

In [25]:
vectorizer.get_feature_names()

[u'000',
 u'056',
 u'07',
 u'10',
 u'100',
 u'1072',
 u'10an',
 u'10h',
 u'11',
 u'11eme',
 u'11eu50',
 u'11h30',
 u'12',
 u'120',
 u'1200',
 u'12h',
 u'13',
 u'130',
 u'13e',
 u'13eme',
 u'13ieme',
 u'14',
 u'14000',
 u'1400euro',
 u'14eme',
 u'14h',
 u'14h16h',
 u'15',
 u'150',
 u'1500',
 u'15h',
 u'15mn',
 u'16',
 u'16h',
 u'1700',
 u'17h',
 u'17h30',
 u'18',
 u'180',
 u'1821',
 u'18h',
 u'1930',
 u'1990',
 u'19h',
 u'1er',
 u'1ere',
 u'1ou2',
 u'20',
 u'200',
 u'2000',
 u'2005',
 u'2007',
 u'2009de',
 u'2010',
 u'2011',
 u'2013',
 u'2015',
 u'2015cesu',
 u'2016',
 u'20h',
 u'20hoo',
 u'22',
 u'22h',
 u'23',
 u'231e47',
 u'2400e',
 u'25',
 u'250',
 u'27',
 u'2an',
 u'2eme',
 u'2h',
 u'2j',
 u'30',
 u'300',
 u'31',
 u'32',
 u'32h',
 u'35',
 u'350',
 u'35h',
 u'360',
 u'366de',
 u'36h',
 u'38',
 u'40',
 u'400',
 u'4000',
 u'40h',
 u'43',
 u'44',
 u'450repa',
 u'48h',
 u'49',
 u'4d',
 u'4k',
 u'50',
 u'500',
 u'5000',
 u'5x8',
 u'65',
 u'650',
 u'6e',
 u'70',
 u'75',
 u'7h30',
 u'7h33'

In [26]:
transformer = TfidfTransformer()

In [27]:
tfidf = transformer.fit_transform(X)

In [28]:
type(tfidf)

scipy.sparse.csr.csr_matrix

# Décomposition SVD

In [29]:
u, s0, vt = linalg.svds(tfidf, k = 100)

In [30]:
s0

array([  2.57146813,   2.5749714 ,   2.58404548,   2.58896826,
         2.59859889,   2.61163784,   2.62669241,   2.62798311,
         2.64688868,   2.65586748,   2.6575082 ,   2.66627157,
         2.68936588,   2.69680495,   2.69751705,   2.70966826,
         2.71606475,   2.72206623,   2.73256932,   2.75176018,
         2.76227947,   2.78775826,   2.79955783,   2.80708233,
         2.81324726,   2.83756297,   2.84954823,   2.8618314 ,
         2.86746847,   2.8771095 ,   2.89056279,   2.90397084,
         2.91431147,   2.92502701,   2.93429036,   2.94774149,
         2.95325138,   2.96361718,   2.97196323,   2.98002388,
         3.00646993,   3.01987358,   3.04300127,   3.04711669,
         3.0624829 ,   3.07175535,   3.09195055,   3.12088696,
         3.12361967,   3.14653827,   3.1688266 ,   3.18724031,
         3.19942954,   3.22386205,   3.24122688,   3.25956521,
         3.26700568,   3.27789643,   3.3014275 ,   3.31288518,
         3.33314194,   3.34585797,   3.36441786,   3.39

In [31]:
m = len([w for w in s0 if w > s0.mean()])

In [32]:
m

33

In [33]:
U, s, Vt = linalg.svds(tfidf, k = m)

In [52]:
U

array([[-0.00962769, -0.00808448, -0.00471034, ...,  0.00973188,
         0.00414989,  0.02820652],
       [-0.01142095,  0.00576485,  0.0026747 , ..., -0.00613008,
        -0.00348207,  0.01641637],
       [-0.00100378,  0.00057899,  0.00487642, ..., -0.00719714,
        -0.01077537,  0.00422809],
       ..., 
       [ 0.01038293, -0.00330607,  0.00872069, ..., -0.01124659,
        -0.01769175,  0.00746201],
       [ 0.01948334, -0.00604859, -0.01845485, ..., -0.00122918,
        -0.00023253,  0.00229288],
       [-0.01057087, -0.00240792,  0.00619544, ..., -0.00372243,
        -0.00734038,  0.0089332 ]])

In [34]:
sd = numpy.zeros((m,m))

In [35]:
numpy.fill_diagonal(sd, s)

In [36]:
sd

array([[  3.48701303,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   3.54607462,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   3.56042667, ...,   0.        ,
          0.        ,   0.        ],
       ..., 
       [  0.        ,   0.        ,   0.        , ...,   5.5368717 ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          7.13642537,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,  11.22602962]])

In [37]:
VS = numpy.dot(numpy.transpose(Vt),sd)

In [38]:
VS.shape

(6809L, 33L)

### Clustering K-means

In [39]:
km_model = KMeans(n_clusters=20)

In [40]:
km_model.fit(VS)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=20, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [41]:
clustering = collections.defaultdict(list)

In [42]:
for idx, label in enumerate(km_model.labels_):
    clustering[label].append(idx)

In [43]:
d ={}

In [44]:
for label in clustering :
    d[label] = []
    for i in clustering[label]:
        d[label] = d[label]+[vectorizer.get_feature_names()[i]]

In [45]:
#writer = csv.writer(open('api_clusters.csv','wb'))
#for key, value in d.items():
#    writer.writerow([key, value])

In [46]:
len(d)

20

### Printing du résultat

In [47]:
for k in range(0,len(d)):
    if len(clustering[k]) <= 30 :
        print '---------'+'cluster '+ str(k)
        for i in  clustering[k]:
            print vectorizer.get_feature_names()[i]

---------cluster 1
tre
---------cluster 2
ambiance
---------cluster 3
bon
---------cluster 4
être
---------cluster 5
salaire
---------cluster 6
avoir
---------cluster 7
travail
---------cluster 9
entreprise
---------cluster 10
beaucoup
---------cluster 11
interessant
---------cluster 12
dynamique
jeun
---------cluster 13
autonomie
---------cluster 14
equipe
---------cluster 15
restaurant
ticket
---------cluster 16
mission
---------cluster 17
humain
taille
---------cluster 18
horaire
---------cluster 19
evolution
possibilite


In [48]:
for k in range(0,len(d)):
    print 'cluster '+ str(k) + '----'+str(len(clustering [k]))


cluster 0----91
cluster 1----1
cluster 2----1
cluster 3----1
cluster 4----1
cluster 5----1
cluster 6----1
cluster 7----1
cluster 8----6696
cluster 9----1
cluster 10----1
cluster 11----1
cluster 12----2
cluster 13----1
cluster 14----1
cluster 15----2
cluster 16----1
cluster 17----2
cluster 18----1
cluster 19----2


In [49]:
for i in  clustering[15]:
    print vectorizer.get_feature_names()[i]

restaurant
ticket


In [50]:
for i in  clustering[5]:
    print vectorizer.get_feature_names()[i]

salaire


In [51]:
for i in  clustering[34]:
    print vectorizer.get_feature_names()[i]

### LDA model using Gensim

In [53]:
from gensim import corpora, models, similarities

In [54]:
id2word = corpora.Dictionary(df_fr['pros_lemmas'])

In [55]:
print(id2word.token2id)

{u'': 112, u'four': 6143, u'competencesexige': 3545, u'insolit': 3748, u'devenir': 3739, u'paternaliste': 5157, u'originalite': 6456, u'lors': 1335, u'electricite': 1862, u'travailentreprise': 2966, u'dela': 659, u'tutorat': 6746, u'bouillonant': 3268, u'regional': 5787, u'tranversale': 3628, u'medicament': 2542, u'metrologie': 4839, u'techniquesle': 6448, u'scholar': 4698, u'clientele': 487, u'humeur': 206, u'businessle': 4964, u'apercu': 612, u'directement': 2401, u'produitsde': 3753, u'tirer': 1234, u'second': 6057, u'recyclage': 3194, u'internetexcellente': 5681, u'peage': 4275, u'predictif': 6570, u'fusionner': 3575, u'responsabilitela': 4079, u'reporter': 3975, u'divertissement': 3831, u'manageur': 2968, u'specialise': 4966, u'appartenanceproces': 2222, u'connaissance': 1165, u'naturel': 2570, u'inter-equipe': 1660, u'regroupement': 4664, u'appengine': 5880, u'controlee': 6181, u'elle-meme': 4491, u'fourchette': 113, u'projection': 1203, u'sympal': 6870, u'rigoureux': 1320, u'uni

In [56]:
corpus = [id2word.doc2bow(text) for text in df_fr['pros_lemmas']]

In [59]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=100, update_every=1, chunksize=10000, passes=1)

In [60]:
lda.show_topics()

[(69,
  u'0.032*bon + 0.032*travail + 0.023*tre + 0.022*avoir + 0.021*ambiance + 0.016*pouvoir + 0.012*apprendre + 0.010*beaucoup + 0.010*employe + 0.009*sympa'),
 (58,
  u'0.059*\xeatre + 0.035*bon + 0.033*avoir + 0.017*tre + 0.017*travail + 0.013*bien + 0.011*agreable + 0.010*ambiance + 0.009*collegue + 0.009*beaucoup'),
 (22,
  u'0.055*avoir + 0.022*travail + 0.014*autonomie + 0.014*equipe + 0.013*ambiance + 0.013*bon + 0.011*formation + 0.010*grand + 0.009*evolution + 0.008*jeun'),
 (61,
  u'0.048*avoir + 0.030*\xeatre + 0.021*travailler + 0.021*beaucoup + 0.017*projet + 0.014*apprendre + 0.014*entreprise + 0.014*tre + 0.013*bon + 0.011*travail'),
 (48,
  u'0.023*avoir + 0.021*bon + 0.021*entreprise + 0.020*\xeatre + 0.018*equipe + 0.016*travail + 0.013*salaire + 0.011*" + 0.011*environnement + 0.010*agreable'),
 (82,
  u'0.047*bon + 0.044*\xeatre + 0.020*interessant + 0.020*avoir + 0.017*entreprise + 0.013*projet + 0.012*tre + 0.011*collegue + 0.009*ambiance + 0.008*equipe'),
 (35